In [33]:
import os 
import re
import string
import pandas as pd
import numpy 
import dagshub
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import scipy.sparse

In [34]:
import warnings
warnings.filterwarnings("ignore")

In [35]:
CONFIG = {
    "test_size" : 0.2,
    "data_path" : "IMDB Dataset.csv",
    "mlflow_tracking_uri" : "https://dagshub.com/codewithkaran-21/Capstone-Project-MLOPS.mlflow",
    "dagshub_repo_owner" :  "codewithkaran-21",
    "dagshub_repo_name" :  "Capstone-Project-MLOPS",
    "experimnent_name" : "BOW vs tfIDF"
}

In [36]:
mlflow.set_tracking_uri(CONFIG["mlflow_tracking_uri"])
dagshub.init(repo_owner=CONFIG['dagshub_repo_owner'] , repo_name=CONFIG['dagshub_repo_name'],mlflow=True)
mlflow.set_experiment(CONFIG['experimnent_name'])

Initialized MLflow to track repo "codewithkaran-21/Capstone-Project-MLOPS"

Repository codewithkaran-21/Capstone-Project-MLOPS initialized!

2025/11/05 11:58:26 INFO mlflow.tracking.fluent: Experiment with name 'BOW vs tfIDF' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/a059b350a4dc485ba1f771dccc2a3c8e', creation_time=1762324107199, experiment_id='2', last_update_time=1762324107199, lifecycle_stage='active', name='BOW vs tfIDF', tags={}>

In [37]:
def lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

def remove_stop_words(text):
    stop_words = set(stopwords.words("english"))
    return " ".join([word for word in text.split() if word not in stop_words])

def removing_numbers(text):
    return ''.join([char for char in text if not char.isdigit()])

def lower_case(text):
    return text.lower()

def removing_punctuations(text):
    return re.sub(f"[{re.escape(string.punctuation)}]", ' ', text)

def removing_urls(text):
    return re.sub(r'https?://\S+|www\.\S+', '', text)

def normalize_text(df):
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f"Error during text normalization: {e}")
        raise

In [38]:
def load_data(file_path):
    try:
        df = pd.read_csv(file_path)
        df = normalize_text(df)
        df = df[df['sentiment'].isin(['positive' , 'negative'])]
        df['sentiment'] = df['sentiment'].replace({'negative' : 0 , 'positive' : 1}).infer_objects(copy=False)
        return df
    except Exception as e:
        print(f"Error occured loading data {e}")
        raise

In [39]:
VECTORIZERS = {
    "BoW" : CountVectorizer(),
    "TF-IDF" : TfidfVectorizer()
}

In [40]:
ALGORITHMS = {
    "LogisticRegression"  :LogisticRegression(),
    "RandomForestClassifier" : RandomForestClassifier(),
    "GradientBoostingClassifier" : GradientBoostingClassifier(),
    "MultinomialNB" : MultinomialNB(),
    "xgboost" : XGBClassifier()
}

In [45]:
def log_model_params(algo_name , model):
    params_to_log = {}
    if algo_name == "LogisticRegression":
        params_to_log["C"] = model.C
    elif algo_name == "RandomForestClassifier":
        params_to_log["n_estimators"] = model.n_estimators 
        params_to_log["max_depth"] = model.max_depth
    elif algo_name == "GradientBoostingClassifier":
        params_to_log["n_estimators"] = model.n_estimators
        params_to_log["learning_rate"] = model.learning_rate
        params_to_log["max_depth"] = model.max_depth
    elif algo_name == "MultinomialNB":
        params_to_log["alpha"] = model.alpha
    elif algo_name == "xgboost":
        params_to_log["n_estimators"] = model.n_estimators
        params_to_log["learning_rate"] = model.learning_rate
    
    mlflow.log_params(params_to_log)

In [46]:
def train_and_evaluate(df):
    with mlflow.start_run(run_name="All-Experiment") as parent_run:
        for algo_name , ALGORITHM in ALGORITHMS.items():
            for vec , vectorizer in VECTORIZERS.items():
                with mlflow.start_run(run_name=f"{algo_name} with {vec}" , nested=True) as child_run:
                    X = vectorizer.fit_transform(df['review'])
                    y = df['sentiment']
                    X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=CONFIG['test_size'] , random_state=42)
                    mlflow.log_params({
                        "vectorizer" : vec,
                        "algorithm" : algo_name,
                        "testsize" : CONFIG['test_size']}
                    )

                    model = ALGORITHM
                    model.fit(X_train , y_train)

                    log_model_params(algo_name , model)

                    y_pred = model.predict(X_test)

                    metrics = {
                        "Accuracy"  : accuracy_score(y_test , y_pred),
                        "precision"  :precision_score(y_test , y_pred),
                        "reacll"  : recall_score(y_test , y_pred),
                        "f1-score" : f1_score(y_test , y_pred)

                    }

                    mlflow.log_metrics(metrics)

                    input_example = X_test[:5] if not scipy.sparse.issparse(X_test) else X_test[:5].toarray()
                    mlflow.sklearn.log_model(model , "model" , input_example=input_example)
                    print(f"Algorithm : {algo_name}, Vectorizer : {vec}")
                    print(f"Metrics : {metrics}")



In [47]:
df = load_data(file_path=CONFIG["data_path"])
train_and_evaluate(df)

2025/11/05 12:11:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run LogisticRegression with BoW at: https://dagshub.com/codewithkaran-21/Capstone-Project-MLOPS.mlflow/#/experiments/2/runs/a15881ea20974455be4a881f820ea82c
🧪 View experiment at: https://dagshub.com/codewithkaran-21/Capstone-Project-MLOPS.mlflow/#/experiments/2
🏃 View run All-Experiment at: https://dagshub.com/codewithkaran-21/Capstone-Project-MLOPS.mlflow/#/experiments/2/runs/e12fd917c67648a084e9db21323b1896
🧪 View experiment at: https://dagshub.com/codewithkaran-21/Capstone-Project-MLOPS.mlflow/#/experiments/2


RestException: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}

RAW RESPONSE:
 {"success":true,"status":200,"health":"100%"}
{
    "success": true,
    "status": 200,
    "health": "100%"
}


{"id":82,"url":"https://www.tvmaze.com/shows/82/game-of-thrones","name":"Game of Thrones","type":"Scripted","language":"English","genres":["Drama","Adventure","Fantasy"],"status":"Ended","runtime":60,"averageRuntime":61,"premiered":"2011-04-17","ended":"2019-05-19","officialSite":"http://www.hbo.com/game-of-thrones","schedule":{"time":"21:00","days":["Sunday"]},"rating":{"average":8.9},"weight":99,"network":{"id":8,"name":"HBO","country":{"name":"United States","code":"US","timezone":"America/New_York"},"officialSite":"https://www.hbo.com/"},"webChannel":null,"dvdCountry":null,"externals":{"tvrage":24493,"thetvdb":121361,"imdb":"tt0944947"},"image":{"medium":"https://static.tvmaze.com/uploads/images/medium_portrait/498/1245274.jpg","original":"https://static.tvmaze.com/uploads/images/original_untouched/498/1245274.jpg"},"summary":"<p>Based on the bestselling book series <i>A Song of Ice and Fire</i> by George R.R. Martin, this sprawling new HBO drama is set in a world where summers spa

{"status":"OK","request_id":"ea97217d-efea-44a3-bb95-907d20026235","data":[{"location":"United States","job_title":"Software Developer","company":"Amazon","min_salary":174943.22,"max_salary":264149.97,"median_salary":212601.19,"min_base_salary":121828.93,"max_base_salary":165003.3,"median_base_salary":141782.14,"min_additional_pay":53114.29,"max_additional_pay":99146.67,"median_additional_pay":70819.05,"salary_period":"YEAR","salary_currency":"USD","salary_count":1075,"confidence":"CONFIDENT"}]}
